In [2]:
import sys
sys.path.append("/opt/ml/code/")

In [3]:
from datasets import load_from_disk
import torch
from transformers import AutoTokenizer
from transformers import BertModel, BertPreTrainedModel, AdamW, TrainingArguments, get_linear_schedule_with_warmup

import random

import model_encoder

## Load 데이터셋

In [4]:
from datasets import load_dataset, load_from_disk

dataset = load_from_disk('../../data/train_dataset')

## Load 모델, 토크나이저

In [18]:
p_encoder = model_encoder.BertEncoder.from_pretrained('../encoders/p_encoder_neg')
q_encoder = model_encoder.BertEncoder.from_pretrained('../encoders/q_encoder_neg')

tokenizer = AutoTokenizer.from_pretrained('klue/bert-base')

if torch.cuda.is_available():
  p_encoder.cuda()
  q_encoder.cuda()

## 검증

In [23]:
valid_corpus = list(set([example['context'] for example in dataset['validation']]))#[:10]
sample_idx = random.choice(range(len(dataset['validation'])))
query = dataset['validation'][sample_idx]['question']
ground_truth = dataset['validation'][sample_idx]['context']

if not ground_truth in valid_corpus:
  valid_corpus.append(ground_truth)

print(query)
print(ground_truth, '\n\n')


뱌쿠롄과 레이무 일행은 어떤 철학 때문에 싸우게 되었나?
마계의 일부 '법계' 에 봉인되어 있던, 원래 인간이었던 마법사이다 . 『성련선』의 사건은 뱌쿠렌을 따르면 요괴들이 그녀의 봉인을 풀려고 하고 일어난 것이다.\n전설의 승려인 묘우렌의 누이인 그녀 자신도 늙은 뒤 동생 묘우렌으로부터 법력을 배웠다. 묘우렌의 죽음을 계기로 죽음을 극도로 무서워하게 되고 법력이 아닌 요력, 마력과 같은 종류의 계략에 의한 회춘의 힘을 손에 넣었다. 승려로서 사람들의 신봉을 받고 겉으로는 요괴 퇴치를 의뢰하는 한편, 회춘의 요력을 유지하기 위해서 뒤에서는 요괴를 도왔다. 요괴를 돕는 동안에 요괴들의 안타까운 과거를 알고 아욕이 아닌 본심에서 요괴들의 힘이 될 수 있었으면 하게 됐다. 그 뒤 뱌쿠렌이 요괴를 도운 일이 인간들에게 발각되자, 그녀는 악마처럼 여겨졌고 마계에 봉인되고 말았다.\n봉인에서 수백 년 하쿠레이 레이무 일행이 모은 비창의 파편에 의한 봉인에서 풀려났으나 뱌쿠렌의 "인간과 요괴의 완전한 평등" 이라는 사상이 레이무 일행에게 받아들여지지 않아 전투를 하였다.\n그 뒤 환상향의 인간의 마을 근처에 성련선이 변화한 "명련사" 라는 절을 건립하고 요괴들을 널리 받아들인다. 보선(성련선)에서 달라진 사찰은 길하다고 인간 사이에서도 평판이 되고 다시 인간들의 신봉도 받게 됐다 .\n엔딩에 따라서는 성련선을 유람선으로 운영하는 모습이 보인。\n『신령묘』에서는 이름만 등장. 레이무에게 "(종교적인 의미에서)장사 라이벌" 로 알려졌다 . 




In [24]:
with torch.no_grad():
  p_encoder.eval()
  q_encoder.eval()

  q_seqs_val = tokenizer([query], padding="max_length", truncation=True, return_tensors='pt').to('cuda')
  # q_seqs_val = tokenizer(['해바라기는 무슨 꽃일까?'], padding="max_length", truncation=True, return_tensors='pt').to('cuda')
  q_emb = q_encoder(**q_seqs_val).to('cpu')  #(num_query, emb_dim)

  p_embs = []
  for p in valid_corpus:
    p = tokenizer(p, padding="max_length", truncation=True, return_tensors='pt').to('cuda')
    p_emb = p_encoder(**p).to('cpu').numpy()
    p_embs.append(p_emb)

p_embs = torch.Tensor(p_embs).squeeze()  # (num_passage, emb_dim)

print(p_embs.size(), q_emb.size())

torch.Size([235, 768]) torch.Size([1, 768])


In [25]:
dot_prod_scores = torch.matmul(q_emb, torch.transpose(p_embs, 0, 1))
print(dot_prod_scores.size())

rank = torch.argsort(dot_prod_scores, dim=1, descending=True).squeeze()

torch.Size([1, 235])


In [26]:
k = 5
print("[Search query]\n", query, "\n")
print("[Ground truth passage]")
print(ground_truth, "\n")

for i in range(k):
  print("Top-%d passage with score %.4f" % (i+1, dot_prod_scores.squeeze()[rank[i]]))
  print(valid_corpus[rank[i]])

[Search query]
 뱌쿠롄과 레이무 일행은 어떤 철학 때문에 싸우게 되었나? 

[Ground truth passage]
마계의 일부 '법계' 에 봉인되어 있던, 원래 인간이었던 마법사이다 . 『성련선』의 사건은 뱌쿠렌을 따르면 요괴들이 그녀의 봉인을 풀려고 하고 일어난 것이다.\n전설의 승려인 묘우렌의 누이인 그녀 자신도 늙은 뒤 동생 묘우렌으로부터 법력을 배웠다. 묘우렌의 죽음을 계기로 죽음을 극도로 무서워하게 되고 법력이 아닌 요력, 마력과 같은 종류의 계략에 의한 회춘의 힘을 손에 넣었다. 승려로서 사람들의 신봉을 받고 겉으로는 요괴 퇴치를 의뢰하는 한편, 회춘의 요력을 유지하기 위해서 뒤에서는 요괴를 도왔다. 요괴를 돕는 동안에 요괴들의 안타까운 과거를 알고 아욕이 아닌 본심에서 요괴들의 힘이 될 수 있었으면 하게 됐다. 그 뒤 뱌쿠렌이 요괴를 도운 일이 인간들에게 발각되자, 그녀는 악마처럼 여겨졌고 마계에 봉인되고 말았다.\n봉인에서 수백 년 하쿠레이 레이무 일행이 모은 비창의 파편에 의한 봉인에서 풀려났으나 뱌쿠렌의 "인간과 요괴의 완전한 평등" 이라는 사상이 레이무 일행에게 받아들여지지 않아 전투를 하였다.\n그 뒤 환상향의 인간의 마을 근처에 성련선이 변화한 "명련사" 라는 절을 건립하고 요괴들을 널리 받아들인다. 보선(성련선)에서 달라진 사찰은 길하다고 인간 사이에서도 평판이 되고 다시 인간들의 신봉도 받게 됐다 .\n엔딩에 따라서는 성련선을 유람선으로 운영하는 모습이 보인。\n『신령묘』에서는 이름만 등장. 레이무에게 "(종교적인 의미에서)장사 라이벌" 로 알려졌다 . 

Top-1 passage with score 209.2675
연방군 수뇌는 「지온에 시간을 주면, 다시 한번 솔라 레이를 사용할 기회를 주게 된다」하며 잔존 함대에 아 바오아 쿠 재진격을 명령했다.\n12월 31일, 연방군은 전투함 마할,화이트 베이스를 중심으로 잔존 함대를 재편성하고 아 바오아 쿠로의 진격을 재개했다. 지온 군도 솔라 레이의 위력으로 사기는 오르고